In [1]:
import os
import pandas as pd
from all_class.get_df import Get_df
from all_class.output_HC import Output
from all_class.v_nucl import V_nucl
from all_class.get_pgen import Get_pgen
from all_class.output_MC import Output_MC

 IMPORTING THE DATASET 

In [2]:
data_directory="../Data/quake_old/all_days"
relevant_days = ["D0","D7"]
subject_days = ["D-5","D-3","D0","D4","D7","D9","D11"]
patients=["Patient_1","Patient_2","Patient_3","Patient_4","Patient_5"]

In [2]:
data_mouse="../../GFRAL/topo_1/only_h"
mouse_days=["D17","D25","D39","D46","D60","D67"]

In [3]:
df_mouse={}
for day in mouse_days[:]:
    df_mouse[day]=pd.read_table(data_mouse+"/{}/alignments.tsv".format(day),sep="\t")

In [3]:
df={}
for p in patients:
    df[p]={}
    for day in subject_days:
        df[p][day]=pd.read_table(data_directory+"/df_{}_{}.tsv".format(p,day),sep="\t")
        df[p][day].drop("family",axis=1,inplace=True)

In [5]:
data=df_mouse["D67"]

PRODUCE A DATAFRAME WITH ALL THE FEATURES

In [5]:
start = Get_df(data)
df_read = start.make()
mult_1, uniq_nucl = start.multiplicity()

OUTPUT: LIST OF HIGH CONFIDENCE BINDING SEQUENCES

In [6]:
threshold=0.000961 #inferred from min fdr of 5%
cluster_min=10 #min size to cluster the sequences

In [7]:
get_output = Output(df_read)
Binders = get_output.cluster(threshold,cluster_min)
df_read["Hits_HC"]=df_read.aaSeqCDR3.isin(Binders["aaSeqCDR3"])

 FOR V USAGE

In [8]:
get_vnucl = V_nucl(data)
v_nucl = get_vnucl.make_vnucl()

TO COMPUTE PGEN (For one sequence or a column of DataFrame of sequences)

In [9]:
get_pgen = Get_pgen()
Pgen = get_pgen.compute_pgen("CARATAVVRGIINHFDYW") #To compute the Pgen of 1 seq
Pgen_parallel=get_pgen.apply_parallel(df_read["aaSeqCDR3"]) # " of a column of df using 10 CPU
df_read["Pgen"]=Pgen_parallel

OUTPUT: LIST OF MEDIUM CONFIDENCE BINDING SEQUENCES

In [10]:
output_MC=Output_MC(df_read)
output_MC.get_pvalue(uniq_nucl)
alpha=0.005 #value to set, %of estimated false positive 
Weak_binders=output_MC.BH_procedure(uniq_nucl,alpha)
df_read["Hits_MC"]=df_read.aaSeqCDR3.isin(Weak_binders["aaSeqCDR3"])

EXPORT THE RESULTS

In [12]:
df_read.to_csv("output/df_read_test.tsv",sep="\t",index=False)